In [139]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('./data/311_calls_small_almost_cleaned.csv')
show_progress = lambda: df['Incident Zip'].isna().sum()

place_desc = ['Borough', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type']

show_progress()

/var/folders/b3/cjvms6c10k77tgv6k_rw3jbh0000gn/T/ipykernel_46729/3730440412.py:5: DtypeWarning: Columns (31,32,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/311_calls_small_almost_cleaned.csv')


89526

In [140]:
df[place_desc].head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,BROOKLYN,11226.0,2709 CLARENDON ROAD,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
1,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BROOKLYN,11211.0,201 SOUTH 4 STREET,SOUTH 4 STREET,SOUTH 5 PLACE,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
3,MANHATTAN,10032.0,533 WEST 158 STREET,WEST 158 STREET,AMSTERDAM AVENUE,BROADWAY,NaN,NaN,ADDRESS,NEW YORK,NaN,NaN
4,STATEN ISLAND,10303.0,6 MACORMAC PLACE,MACORMAC PLACE,NORTHFIELD AVENUE,HOLLAND AVENUE,NaN,NaN,ADDRESS,STATEN ISLAND,NaN,NaN


In [141]:
bu = df['Borough'] == 'Unspecified'
df.loc[bu, 'Borough'] = np.nan

In [142]:
df['Borough'].unique()

array(['BROOKLYN', nan, 'MANHATTAN', 'STATEN ISLAND', 'BRONX', 'QUEENS'],
      dtype=object)

In [143]:
df['Borough'].isna().sum()

18600

In [144]:
for col in place_desc:
    print(f'{col:<22}: {(df[col].isna().sum() / df.shape[0]):.2%}')

Borough               : 1.86%
Incident Zip          : 8.95%
Incident Address      : 14.45%
Street Name           : 14.45%
Cross Street 1        : 46.77%
Cross Street 2        : 46.97%
Intersection Street 1 : 87.54%
Intersection Street 2 : 87.56%
Address Type          : 8.37%
City                  : 8.91%
Landmark              : 98.91%
Facility Type         : 72.09%


In [145]:
is_brooklyn = lambda: df['Borough'] == 'BROOKLYN'   # "live" masks
is_staten = lambda: df['Borough'] == 'STATEN ISLAND'
is_bronx = lambda: df['Borough'] == 'BRONX'
is_manhattan = lambda: df['Borough'] == 'MANHATTAN'
is_queens = lambda: df['Borough'] == 'QUEENS'

In [146]:
df.loc[is_brooklyn(), place_desc].sort_values(by='Street Name').head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
396356,BROOKLYN,NaN,481-483 .DEKALB AVE,.DEKALB AVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455159,BROOKLYN,NaN,4821 1 AVENUE,1 AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551777,BROOKLYN,11232.0,4000 1 AVENUE,1 AVENUE,40 STREET,41 STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,Precinct
170575,BROOKLYN,11220.0,5401 1 AVENUE,1 AVENUE,54 STREET,55 STREET,54 STREET,55 STREET,NaN,BROOKLYN,1 AVENUE,NaN
669247,BROOKLYN,11220.0,5601 1 AVENUE,1 AVENUE,56 STREET,57 STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN


In [147]:
def uniform_names(x):
    pt1 = re.compile(r'\ ST\.*$')
    pt2 = re.compile(r'\ AVE\.*$')
    pt3 = re.compile(r'\ RD\.*$')
    pt4 = re.compile(r' \ PL\.*$')
    pt5 = re.compile(r'\w{3}')
    try:
        if re.search(pt1, x):
            return re.sub(pt1, ' STREET', x).upper()
        elif re.search(pt2, x):
            return re.sub(pt2, ' AVENUE', x).upper()
        elif re.search(pt3, x):
            return re.sub(pt3, ' ROAD', x).upper()
        elif re.search(pt4, x):
            return re.sub(pt4, ' PLACE', x).upper()
        elif not re.search(pt5, x):
            return np.nan
    except:
        return x
    return x


In [148]:
street_cols = ['Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2']
for col in street_cols:
    df[col] = df[col].apply(lambda x: uniform_names(x))
    

In [149]:
df[place_desc].head()

,Borough,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type
0,BROOKLYN,11226.0,2709 CLARENDON ROAD,CLARENDON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BROOKLYN,11211.0,201 SOUTH 4 STREET,SOUTH 4 STREET,SOUTH 5 PLACE,ROEBLING STREET,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN
3,MANHATTAN,10032.0,533 WEST 158 STREET,WEST 158 STREET,AMSTERDAM AVENUE,BROADWAY,NaN,NaN,ADDRESS,NEW YORK,NaN,NaN
4,STATEN ISLAND,10303.0,6 MACORMAC PLACE,MACORMAC PLACE,NORTHFIELD AVENUE,HOLLAND AVENUE,NaN,NaN,ADDRESS,STATEN ISLAND,NaN,NaN


In [150]:
df.dropna(subset=place_desc, how='all', inplace=True)

In [151]:
show_progress()

75241

In [152]:
dontuse = ['Address Type', 'City', 'Landmark', 'Facility Type']
subset = [col for col in place_desc if col not in dontuse]

def find_unique_zips(boro_mask, attr_list):
    tmp = df.loc[boro_mask, place_desc].copy()   # snapshot per borough
    tmp = tmp.drop_duplicates(subset=subset, keep='first')  # one instance per combination
    tmp = tmp.dropna(subset=attr_list, how='any')  # drop any null value
    
    grouped = tmp.groupby(attr_list)['Incident Zip'].unique()  # pd.Series of np.arrays
    grouped = grouped.apply(lambda x: x[~ np.isnan(x)] if np.isnan(x.sum()) else x)  # if it has one np.nan
    flattened = ( grouped.apply(lambda x: x.flatten()[0] if # extract the float value..
                    (isinstance(x, np.ndarray) and (x.size > 0)) 
                    else np.nan).dropna()  # ...and drop any null values remaining
                )
    return flattened

In [153]:
street_attrs = ['Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
 'Intersection Street 1', 'Intersection Street 2']

from itertools import combinations
for pair in list(combinations(street_attrs, 2)):
    old = show_progress()
    attr_list = ['Borough'] + list(pair)
    for boro_mask in [is_brooklyn, is_bronx, is_queens, is_manhattan, is_staten]:
        flat = find_unique_zips(boro_mask(), attr_list)
        flat = flat.reset_index().set_index(attr_list) 
        if not flat.dropna().empty:
            df = df.set_index(attr_list)   # align the indexes
            df.update(flat)
            df = df.reset_index()
    print(f'{pair}: {old - show_progress()} zips updated')
        

('Incident Address', 'Street Name'): 22958 zips updated
('Incident Address', 'Cross Street 1'): 8 zips updated
('Incident Address', 'Cross Street 2'): 1 zips updated
('Incident Address', 'Intersection Street 1'): 0 zips updated
('Incident Address', 'Intersection Street 2'): 0 zips updated
('Street Name', 'Cross Street 1'): 390 zips updated
('Street Name', 'Cross Street 2'): 33 zips updated
('Street Name', 'Intersection Street 1'): 16 zips updated
('Street Name', 'Intersection Street 2'): 1 zips updated
('Cross Street 1', 'Cross Street 2'): 1164 zips updated
('Cross Street 1', 'Intersection Street 1'): 2709 zips updated
('Cross Street 1', 'Intersection Street 2'): 32 zips updated
('Cross Street 2', 'Intersection Street 1'): 53 zips updated
('Cross Street 2', 'Intersection Street 2'): 1316 zips updated
('Intersection Street 1', 'Intersection Street 2'): 3091 zips updated


In [154]:
show_progress()

43469

In [156]:
df.to_csv('./data/311_calls_Take2.csv', index=False)